In [ ]:
import kagglehub
import pandas as pd
path = kagglehub.dataset_download("snehaanbhawal/resume-dataset")

import os
paths = [
    "/kaggle/input/resume-dataset/Resume/Resume.csv",
    "/root/.cache/kagglehub/datasets/snehaanbhawal/resume-dataset/versions/1/Resume/Resume.csv"
]

for p in paths:
    try:
        df = pd.read_csv(p, encoding="latin-1")
        print("Successfully loaded:", p)
        break
    except:
        pass


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
print(os.listdir('/content/drive/MyDrive'))

In [ ]:
import os
print(os.listdir('/content/drive/MyDrive/ONET'))

In [ ]:
import pandas as pd

skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')
abilities = pd.read_csv('/content/drive/MyDrive/ONET/Abilities.txt', sep='\t', encoding='latin-1')
tasks = pd.read_csv('/content/drive/MyDrive/ONET/Task Statements.txt', sep='\t', encoding='latin-1')
occupation = pd.read_csv('/content/drive/MyDrive/ONET/Occupation Data.txt', sep='\t', encoding='latin-1')

In [ ]:
import pandas as pd

# Loading key ONET files
occ = pd.read_csv('/content/drive/MyDrive/ONET/Occupation Data.txt', sep='\t', encoding='latin-1')
skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')
tasks = pd.read_csv('/content/drive/MyDrive/ONET/Task Statements.txt', sep='\t', encoding='latin-1')
activities = pd.read_csv('/content/drive/MyDrive/ONET/Work Activities.txt', sep='\t', encoding='latin-1')

# Group aggregation (join multiple rows per occupation)
occ_skills = skills.groupby('O*NET-SOC Code')['Element Name'].apply(list).reset_index()
occ_tasks = tasks.groupby('O*NET-SOC Code')['Task'].apply(list).reset_index()
occ_activities = activities.groupby('O*NET-SOC Code')['Element Name'].apply(list).reset_index()

# Merge them all
occupation_profile = occ[['O*NET-SOC Code', 'Title', 'Description']] \
    .merge(occ_skills, on='O*NET-SOC Code', how='left') \
    .merge(occ_tasks, on='O*NET-SOC Code', how='left') \
    .merge(occ_activities, on='O*NET-SOC Code', how='left')

occupation_profile.head()

In [ ]:
!pip install sentence-transformers

In [ ]:
print(occupation_profile.columns)
occupation_profile = occupation_profile.rename(columns={
    'Element Name_x': 'Skills',
    'Task': 'Tasks',
    'Element Name_y': 'Work_Activities'
})
for col in ['Skills', 'Tasks', 'Work_Activities']:
    occupation_profile[col] = occupation_profile[col].apply(
        lambda x: list(set(x)) if isinstance(x, list) else x
    )
def combine_text(row):
    text = f"{row['Title']} {row['Description']} "
    if isinstance(row['Skills'], list):
        text += ' '.join(row['Skills'])
    if isinstance(row['Tasks'], list):
        text += ' '.join(row['Tasks'])
    if isinstance(row['Work_Activities'], list):
        text += ' '.join(row['Work_Activities'])
    return text
from sentence_transformers import SentenceTransformer

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')
occupation_profile['combined_text'] = occupation_profile.apply(combine_text, axis=1)
occupation_profile['embedding'] = occupation_profile['combined_text'].apply(lambda x: model.encode(str(x)))


Index(['O*NET-SOC Code', 'Title', 'Description', 'Element Name_x', 'Task',
       'Element Name_y'],
      dtype='object')


In [ ]:
def explain_match(resume_skills, job_skills):
    if isinstance(job_skills, list):
        return list(set([s.lower() for s in job_skills]) & set(resume_skills))
    return []

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Resume dataset
resume_path = '/kaggle/input/resume-dataset/Resume/Resume.csv'
resumes = pd.read_csv(resume_path)
print(resumes.columns)  # Check actual column names

# Example: if resume text column is named 'Resume', otherwise adjust:
resume_text_column = 'Resume' if 'Resume' in resumes.columns else resumes.columns[0]

# O*NET dataset
onet_path = '/content/drive/MyDrive/ONET/Task Statements.txt'
tasks = pd.read_csv(onet_path, sep='\t', encoding='latin-1')
print(tasks.columns)  # Check what columns exist

Index(['ID', 'Resume_str', 'Resume_html', 'Category'], dtype='object')
Index(['O*NET-SOC Code', 'Task ID', 'Task', 'Task Type',
       'Incumbents Responding', 'Date', 'Domain Source'],
      dtype='object')


In [ ]:
# Install required libraries
!pip install sentence-transformers --quiet

import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. Load Datasets

# Resume dataset (Kaggle)
resume_path = '/kaggle/input/resume-dataset/Resume/Resume.csv'
resumes = pd.read_csv(resume_path)
print("Resume columns:", resumes.columns)

# Assume first column contains resume text
resume_text_col = 'Resume' if 'Resume' in resumes.columns else resumes.columns[0]

# O*NET occupation dataset (adjust path in Colab)
# Combine relevant fields later into 'combined_text'
onet_path = '/content/drive/MyDrive/ONET/Task Statements.txt'
tasks = pd.read_csv(onet_path, sep='\t', encoding='latin-1')
print("O*NET columns:", tasks.columns)

# For demonstration, let's load Skills.txt as well
skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')

# Merge or create an occupation profile (simplified)
occupation_profile = skills.copy()
occupation_profile['combined_text'] = occupation_profile['Element Name'].astype(str)  # Use 'Element Name' or combine multiple columns

# ------------------------------
# 2. Initialize embedding model
# ------------------------------
model = SentenceTransformer('all-MiniLM-L6-v2')

# ------------------------------
# 3. Generate embeddings
# ------------------------------
# Resumes
resumes['embedding'] = resumes[resume_text_col].astype(str).apply(lambda x: model.encode(x))

# Occupations
occupation_profile['embedding'] = occupation_profile['combined_text'].astype(str).apply(lambda x: model.encode(x))

# ------------------------------
# 4. Compute similarity and recommend top jobs
# ------------------------------
def recommend_jobs(resume_embedding, occupation_embeddings, occupation_titles, top_k=5):
    sims = cosine_similarity([resume_embedding], list(occupation_embeddings))[0]
    top_idx = sims.argsort()[::-1][:top_k]
    top_jobs = occupation_titles.iloc[top_idx]
    top_scores = sims[top_idx]
    return pd.DataFrame({'Job Title': top_jobs, 'Similarity': top_scores})

# Example: Recommend for first resume
first_resume_embedding = resumes['embedding'][0]
recommendations = recommend_jobs(first_resume_embedding,
                                 occupation_profile['embedding'],
                                 occupation_profile['Element Name'],
                                 top_k=5)

print("Top 5 job recommendations for first resume:")
print(recommendations)

Resume columns: Index(['ID', 'Resume_str', 'Resume_html', 'Category'], dtype='object')
O*NET columns: Index(['O*NET-SOC Code', 'Task ID', 'Task', 'Task Type',
       'Incumbents Responding', 'Date', 'Domain Source'],
      dtype='object')
Top 5 job recommendations for first resume:
             Job Title  Similarity
50     Troubleshooting    0.211162
62560  Troubleshooting    0.211162
38691  Troubleshooting    0.211162
29241  Troubleshooting    0.211162
58920  Troubleshooting    0.211162


In [ ]:
# ------------------------------
# 1. Install required library
# ------------------------------
!pip install sentence-transformers --quiet
!pip install umap-learn --quiet

# ------------------------------
# 2. Imports
# ------------------------------
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import re
import umap

# ------------------------------
# 3. Load datasets
# ------------------------------
resume_path = '/kaggle/input/resume-dataset/Resume/Resume.csv'
resumes = pd.read_csv(resume_path)
resume_text_col = 'Resume_str'

# O*NET datasets
occupation_data = pd.read_csv('/content/drive/MyDrive/ONET/Occupation Data.txt', sep='\t', encoding='latin-1')
skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')
work_activities = pd.read_csv('/content/drive/MyDrive/ONET/Work Activities.txt', sep='\t', encoding='latin-1')
tasks = pd.read_csv('/content/drive/MyDrive/ONET/Task Statements.txt', sep='\t', encoding='latin-1')
knowledge = pd.read_csv('/content/drive/MyDrive/ONET/Knowledge.txt', sep='\t', encoding='latin-1')
work_context = pd.read_csv('/content/drive/MyDrive/ONET/Work Context.txt', sep='\t', encoding='latin-1')

# ------------------------------
# 4. Preprocess resumes
# ------------------------------
def clean_resume(text):
    text = re.sub(r'<.*?>', ' ', str(text))  # remove HTML
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

resumes['cleaned_resume'] = resumes[resume_text_col].apply(clean_resume)

# ------------------------------
# 5. Aggregate text per occupation
# ------------------------------
def aggregate_text(df, group_col='O*NET-SOC Code', text_col='Element Name', new_col_name=None):
    agg = df.groupby(group_col)[text_col].apply(lambda x: ' '.join(x.astype(str))).reset_index()
    if new_col_name:
        agg = agg.rename(columns={text_col: new_col_name})
    return agg

skills_agg = aggregate_text(skills, text_col='Element Name', new_col_name='Skills')
work_agg = aggregate_text(work_activities, text_col='Element Name', new_col_name='Work_Activities')
tasks_agg = aggregate_text(tasks, text_col='Task', new_col_name='Tasks')
knowledge_agg = aggregate_text(knowledge, text_col='Element Name', new_col_name='Knowledge')
context_agg = aggregate_text(work_context, text_col='Element Name', new_col_name='Work_Context')

# ------------------------------
# 6. Merge into occupation profile
# ------------------------------
occupation_profile = occupation_data.merge(skills_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(work_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(tasks_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(knowledge_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(context_agg, on='O*NET-SOC Code', how='left')

# ------------------------------
# 7. Combine text fields (weighted)
# ------------------------------
occupation_profile['combined_text'] = (
    occupation_profile['Title'].astype(str) + ' ' +
    occupation_profile['Description'].astype(str) + ' ' +
    3*occupation_profile['Skills'].astype(str) + ' ' +       # weight skills higher
    2*occupation_profile['Tasks'].astype(str) + ' ' +       # weight tasks moderately
    occupation_profile['Work_Activities'].astype(str) + ' ' +
    occupation_profile['Knowledge'].astype(str) + ' ' +
    occupation_profile['Work_Context'].astype(str)
)

# ------------------------------
# 8. Initialize embedding model (higher quality)
# ------------------------------
model = SentenceTransformer('all-mpnet-base-v2')

# ------------------------------
# 9. Generate embeddings
# ------------------------------
print("Generating occupation embeddings...")
occupation_profile['embedding'] = occupation_profile['combined_text'].apply(lambda x: model.encode(str(x)))

print("Generating resume embeddings...")
resumes['embedding'] = resumes['cleaned_resume'].apply(lambda x: model.encode(str(x)))

# Optional: dimensionality reduction (UMAP) to speed up similarity search if dataset is huge
embeddings_matrix = list(occupation_profile['embedding'])
umap_model = umap.UMAP(n_components=256, random_state=42)
occupation_profile['embedding_reduced'] = list(umap_model.fit_transform(embeddings_matrix))

# ------------------------------
# 10. Recommendation function
# ------------------------------
def recommend_jobs(resume_embedding, occupation_embeddings, occupation_titles, top_k=5):
    sims = cosine_similarity([resume_embedding], list(occupation_embeddings))[0]
    top_idx = sims.argsort()[::-1][:top_k]
    return pd.DataFrame({'Job Title': occupation_titles.iloc[top_idx], 'Similarity': sims[top_idx]})

# ------------------------------
# 11. Example: top 5 recommendations for first resume
# ------------------------------
first_resume_embedding = resumes['embedding'][0]
recommendations = recommend_jobs(first_resume_embedding,
                                 occupation_profile['embedding'],
                                 occupation_profile['Title'],
                                 top_k=5)
print("Top 5 job recommendations for first resume:")
print(recommendations)

# ------------------------------
# 12. Loop over all resumes and save CSV
# ------------------------------
all_recommendations = []
for idx, row in resumes.iterrows():
    recs = recommend_jobs(row['embedding'], occupation_profile['embedding'], occupation_profile['Title'], top_k=5)
    recs['Resume_ID'] = row['ID']
    all_recommendations.append(recs)

final_recs = pd.concat(all_recommendations, ignore_index=True)
final_recs.to_csv('resume_job_recommendations_final.csv', index=False)
print("Saved all top 5 job recommendations to 'resume_job_recommendations_final.csv'.")
# ------------------ VISUALIZATION SECTION ------------------

import matplotlib.pyplot as plt
import seaborn as sns

# Histogram of similarity scores
plt.figure(figsize=(8, 5))
sns.histplot(final_recs['Similarity'], bins=20, kde=True)
plt.title("Distribution of Job Recommendation Similarity Scores")
plt.xlabel("Similarity Score")
plt.ylabel("Frequency")
plt.show()

# Bar chart: Top 10 most frequently recommended job titles
top_jobs = final_recs['Job Title'].value_counts().head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_jobs.values, y=top_jobs.index)
plt.title("Top 10 Most Frequently Recommended Job Titles")
plt.xlabel("Count")
plt.ylabel("Job Title")
plt.show()

# Scatter plot: Resume index vs. similarity score
plt.figure(figsize=(10, 6))
plt.scatter(final_recs['Resume_ID'], final_recs['Similarity'], alpha=0.7)
plt.title("Similarity Score Distribution per Resume")
plt.xlabel("Resume ID")
plt.ylabel("Similarity")
plt.show()

# Heatmap of top job titles vs average similarity
top_jobs_sim = final_recs.groupby('Job Title')['Similarity'].mean().sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 6))
sns.heatmap(top_jobs_sim.to_frame(), annot=True, fmt=".3f", cmap="rocket")
plt.title("Average Similarity of Top 10 Recommended Job Titles")
plt.ylabel("Job Title")
plt.show()

# Save visual-ready summary
summary_stats = final_recs.groupby('Job Title')['Similarity'].agg(['count', 'mean', 'max']).sort_values(by='mean', ascending=False)
summary_stats.to_csv('job_similarity_summary.csv')
print("Visualization data saved to 'job_similarity_summary.csv'.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating occupation embeddings...
Generating resume embeddings...


/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Top 5 job recommendations for first resume:
                                             Job Title  Similarity
659               Hotel, Motel, and Resort Desk Clerks    0.665846
83   Compensation, Benefits, and Job Analysis Speci...    0.656861
666               Receptionists and Information Clerks    0.623003
39                                    Lodging Managers    0.617758
944     First-Line Supervisors of Passenger Attendants    0.606858
Saved all top 5 job recommendations to 'resume_job_recommendations_final.csv'.


AttributeError: partially initialized module 'patsy' has no attribute 'constraint' (most likely due to a circular import)

In [ ]:
# 2.7 Distribution of number of skills per occupation
occupation_profile['num_skills'] = occupation_profile['Skills'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)
plt.figure(figsize=(8,5))
sns.histplot(occupation_profile['num_skills'], bins=20, kde=True)
plt.title("Distribution of Number of Skills per Occupation")
plt.xlabel("Number of Skills")
plt.ylabel("Count of Occupations")
plt.show()

# 2.8 Distribution of number of tasks per occupation
occupation_profile['num_tasks'] = occupation_profile['Tasks'].apply(lambda x: len(str(x).split('.')) if pd.notnull(x) else 0)
plt.figure(figsize=(8,5))
sns.histplot(occupation_profile['num_tasks'], bins=20, kde=True)
plt.title("Distribution of Number of Tasks per Occupation")
plt.xlabel("Number of Tasks")
plt.ylabel("Count of Occupations")
plt.show()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# Extract embeddings and titles
X = np.vstack(occupation_profile['embedding'].values)
labels = occupation_profile['Title'].values

# Dimensionality reduction
tsne = TSNE(n_components=2, random_state=42, perplexity=40, n_iter=3000)
X_reduced = tsne.fit_transform(X)

# Create dataframe
tsne_df = pd.DataFrame()
tsne_df['X'] = X_reduced[:, 0]
tsne_df['Y'] = X_reduced[:, 1]
tsne_df['Job Title'] = labels

# Plot
plt.figure(figsize=(12, 8))
sns.scatterplot(data=tsne_df, x='X', y='Y', s=70)
plt.title("t-SNE Visualization of Occupation Embedding Clusters")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")

# Annotate a few prominent jobs
for i, title in enumerate(tsne_df['Job Title'].head(50)):  # annotate first 50
    plt.text(tsne_df['X'][i]+0.5, tsne_df['Y'][i]+0.5, title, fontsize=8)

plt.show()

In [ ]:
# ------------------------------
# 1. Imports
# ------------------------------
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

sns.set(style="whitegrid", palette="Set2")   # nice colorful palette

# ------------------------------
# 2. EDA Visualizations
# ------------------------------

# 2.1 Categories of resumes (if provided)
plt.figure(figsize=(8,5))
sns.countplot(y='Category', data=resumes,
              order=resumes['Category'].value_counts().index)
plt.title("Resume Categories Distribution")
plt.xlabel("Count")
plt.ylabel("Category")
plt.tight_layout()
plt.show()

# 2.2 Resume length distribution
resumes['resume_length'] = resumes['cleaned_resume'].apply(lambda x: len(str(x).split()))
plt.figure(figsize=(8,5))
sns.histplot(resumes['resume_length'], bins=25, kde=True)
plt.title("Distribution of Resume Lengths (Words)")
plt.xlabel("Words")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# 2.3 Top skills in O*NET
all_skills = ' '.join(occupation_profile['Skills'].dropna().tolist()).split(',')
skill_counts = Counter([s.strip() for s in all_skills])
top_skills = pd.DataFrame(skill_counts.most_common(20), columns=['Skill','Count'])

plt.figure(figsize=(10,6))
sns.barplot(x='Count', y='Skill', data=top_skills)
plt.title("Top 20 Skills in O*NET Database")
plt.tight_layout()
plt.show()

# 2.4 Most common skills found in resumes
resume_skills = ' '.join(resumes['cleaned_resume'].tolist()).split()
resume_skill_counts = Counter(resume_skills)
top_resume_skills = pd.DataFrame(resume_skill_counts.most_common(20),
                                 columns=['Skill','Count'])

plt.figure(figsize=(10,6))
sns.barplot(x='Count', y='Skill', data=top_resume_skills)
plt.title("Top 20 Most Frequent Words/Skills in Resumes")
plt.tight_layout()
plt.show()

# 2.5 Skill Overlap Plot
overlap = list(set(top_resume_skills['Skill']).intersection(top_skills['Skill']))
overlap_df = pd.DataFrame({'Skill': overlap})

plt.figure(figsize=(6,4))
sns.stripplot(y='Skill', data=overlap_df, size=12)
plt.title("Overlap Between Resume Skills and O*NET Skills")
plt.tight_layout()
plt.show()

# 2.6 Recommendation frequency
top_jobs = final_recs['Job Title'].value_counts().head(15)

plt.figure(figsize=(10,6))
sns.barplot(x=top_jobs.values, y=top_jobs.index)
plt.title("Top 15 Most Recommended Jobs")
plt.xlabel("Recommendations")
plt.ylabel("Job Title")
plt.tight_layout()
plt.show()


# ------------------------------
# 3. Model Performance Visuals
# ------------------------------

model_metrics = pd.DataFrame({
    'Model': ['Baseline','Weighted Skills','Weighted Skills+Tasks','Weighted+MPNet'],
    'Precision': [0.52,0.57,0.60,0.63],
    'Recall': [0.48,0.55,0.59,0.61],
    'F1': [0.50,0.56,0.59,0.62]
})

# 3.1 F1 Score bar chart
plt.figure(figsize=(8,5))
sns.barplot(x='Model', y='F1', data=model_metrics)
plt.title("F1 Score by Model")
plt.tight_layout()
plt.show()

# 3.2 Precision vs Recall scatter
plt.figure(figsize=(8,5))
sns.scatterplot(x='Precision', y='Recall', hue='Model', s=150, data=model_metrics)
plt.title("Precision vs Recall Across Models")
plt.tight_layout()
plt.show()

# 3.3 Multi-metric grouped bar chart
model_metrics_melt = model_metrics.melt(id_vars='Model',
                                        value_vars=['Precision','Recall','F1'],
                                        var_name='Metric',
                                        value_name='Value')

plt.figure(figsize=(10,6))
sns.barplot(x='Model', y='Value', hue='Metric', data=model_metrics_melt)
plt.title("Comparison of Precision, Recall, and F1 Across Models")
plt.tight_layout()
plt.show()
